In [21]:
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import json

In [22]:
#!apt-get install -y poppler-utils

In [23]:
# !apt-get install -y tesseract-ocr
#!apt-get install -y tesseract-ocr-script-dev
# !apt-get install -y tesseract-ocr-san

In [24]:
pdf_path = '/kaggle/input/sanskrit-text/Sanskrit_Text (1).pdf'

# Convert the PDF pages to images
pages = convert_from_path(pdf_path)

In [25]:
!tesseract --list-langs

List of available languages (3):
eng
osd
san


In [29]:
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

In [32]:
for i, page in enumerate(pages):
    # Extract the text and bounding box information from the image
    data = pytesseract.image_to_boxes(page)
    
    # Split the data into lines
    lines = data.split('\n')
    
    # Initialize variables to store the current line and its bounding box coordinates
    line = ''
    x_min = float('inf')
    y_min = float('inf')
    x_max = 0
    y_max = 0
    
    # Loop through each line of data
    for j, line_data in enumerate(lines):
        # Split the line data into fields
        fields = line_data.split()
        
        # Check if there are enough fields
        if len(fields) >= 6:
            # Get the character and its bounding box coordinates
            char = fields[0]
            x1, y1, x2, y2 = map(int, fields[1:5])
            
            # Check if the character is not a newline character
            if char != '\n':
                # Add the character to the current line
                line += char
                
                # Update the bounding box coordinates
                x_min = min(x_min, x1)
                y_min = min(y_min, y1)
                x_max = max(x_max, x2)
                y_max = max(y_max, y2)
            else:
                # Check if the current line is not empty
                if line.strip() != '':
                    # Add the bounding box coordinates to the dictionary
                    box_name = f'box{i}_{j}'
                    boxes[box_name] = {
                        'top_left': [x_min, page.height - y_max],
                        'top_right': [x_max, page.height - y_max],
                        'bottom_left': [x_min, page.height - y_min],
                        'bottom_right': [x_max, page.height - y_min]
                    }
                    
                    # Crop the image to the bounding box and save it as a separate JPG file
                    line_image = page.crop((x_min, page.height - y_max, x_max, page.height - y_min))
                    line_image.save(f'{box_name}.jpg')
                    
                    # Reset the current line and its bounding box coordinates
                    line = ''
                    x_min = float('inf')
                    y_min = float('inf')
                    x_max = 0
                    y_max = 0

# Save the bounding box coordinates to a JSON file
with open('boxes.json', 'w') as f:
    json.dump(boxes, f)

In [34]:
import json
from IPython.display import FileLink

# Set the name of the JSON file
json_filename = 'boxes.json'

# Save the bounding box coordinates to a JSON file
with open(json_filename, 'w') as f:
    json.dump(boxes, f)

# Create a download link for the JSON file
FileLink(json_filename)

/kaggle/working/boxes.json